# Preparation

In [ ]:
import csv
import regex
import datetime as dt
from datetime import timedelta
import locale
locale.setlocale(locale.LC_TIME, 'id-ID.UTF-8')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial.distance import cdist
from kneed import KneeLocator
from sklearn import preprocessing
from sklearn.metrics import silhouette_score

from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings("ignore")
#from feature_engine.outlier_removers import Winsorizer

In [ ]:
def check_skew(df_skew, column):
    skew = stats.skew(df_skew[column])
    skewtest = stats.skewtest(df_skew[column])
    plt.title('Distribution of ' + column)
    sns.distplot(df_skew[column], kde=False)
    
    print("{}'s: Skew: {}, : {}".format(column, skew, skewtest))
    return

def norm_minmax(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

def snake_plot(normalised_df_lrfm, df_lrfm_kmeans, df_lrfm_original):

    normalised_df_lrfm = pd.DataFrame(normalised_df_lrfm, 
                                       index=df_lrfm_original.index, 
                                       columns=df_lrfm_original.columns)
    normalised_df_lrfm['Cluster'] = df_lrfm_kmeans['Cluster']

    # Melt data into long format
    df_melt = pd.melt(normalised_df_lrfm.reset_index(), 
                        id_vars=['telp', 'Cluster'],
                        value_vars=['Length', 'Recency', 'Frequency', 'Monetary'], 
                        var_name='Metric', 
                        value_name='Value')

    plt.xlabel('Metric')
    plt.ylabel('Value')
    sns.pointplot(data=df_melt, x='Metric', y='Value', hue='Cluster')
    
    return

def threeD_plot(df_lrfm_kmeans, df_lrfm_original):
    from mpl_toolkits.mplot3d import Axes3D
    
    df_merge = pd.merge(df_lrfm_kmeans, df_lrfm_original['Cluster'], on='telp', how='left')

    x = np.array(df_merge['Recency'])
    y = np.array(df_merge['Frequency'])
    z = np.array(df_merge['Monetary'])

    plt.xlabel('Recency')
    plt.ylabel('Frequency')
    # plt.zlabel('Monetary')  
    ax.legend('Cluster')
    ax.scatter(x,y,z, s=(df_merge['Length']*40), marker="s", c=df_merge["Cluster"], cmap="Paired")


    return

## Import dataset

In [ ]:
df = pd.read_csv("Coret-Coret Marketing_Transaction Details (TA Hammam)_2003-2105.csv")
# df = pd.read_csv("Coret-Coret Marketing_Transaction Details (TA Hammam)_1904-2104.csv")
display(df.info())
df.head(10)

In [ ]:
print("Dataset dimension : " + str(df.shape))
print('Jumlah Customer : ' + str(len(pd.unique(df['telp']))))
print('Jumlah Transaksi : ' + str(len(pd.unique(df['no_order']))))

# Cleaning

## Clean Telp

In [ ]:
#Drop Null
df.dropna(subset=['telp', 'produk', 'name'], inplace=True)

#standarize telp number
df['telp'] = df['telp'].str.replace('-','')
df['telp'] = df['telp'].str.replace(' ','')
df['telp'] = df['telp'].str.replace('+','')
df['telp'] = df['telp'].str.replace('08', '628', 1)
df = df[df['telp'].str.startswith('628')]

#Clean admin no hp & Invalid
df = df.drop(df[df.telp.eq('6281381306699')
                | df.telp.eq('0')].index)
df = df[~df['telp'].str.contains('1234567')] #<--- Belum Bisa

#drop number with invalid character
df = df.drop(df[df.telp.str.contains(r'[^0-9a-zA-Z]')
                | df.telp.str.contains(r'[a-zA-Z]')].index)
                
#drop abnormal telpon length
df = df[df['telp'].str.len() > 10]
df = df[df['telp'].str.len() < 15]

## Clean Name, Channel, Product

In [ ]:
#Clean penyesuaian, test, dan keperluan internal
testing = ['test', 'COBAAA', 'tes123', 'TEXTING', 'penyesuaian', 'penyelamatan', 'stok', 'stock', 'tamu']
df = df[~df['name'].str.contains('|'.join(testing))]

#Clean Old B2B Customer
B2B_cust = ['hotel', 'resto', 'restoran', 'cafe', 'coffee', 'Toko', 'toko', 'Patata', 'Geprek', 'Laziza', 'ayam', 'nasi', 'sego', 'Baksoe', 'Bakso', 'Rismart', 'Nu Mart', 'Warung', 'Rice box','E-Warung', 'ewarung', 'Kedai', 'geprek', 'nelongso', 'pkk', 'pkh']
df = df[~df['name'].str.contains('|'.join(B2B_cust))] 

#Clean B2B Product
B2Bproduct = ['B2B']
df = df[~df['produk'].str.contains('|'.join(B2Bproduct))] 

#Clean Produk Promo & Bundling
Promo = ['PROMO', 'MAM', 'JSM', 'JUARA', 'PESTA', 'GRATIS', 'PAKET', 'SALE', 'PSBB']
df = df[~df['produk'].str.contains('|'.join(Promo))] 

#Clean Based on Channel
channel = ['CL', 'End User', 'Shopee']
df = df[df['Channel'].isin(channel)]

#clean status non delivered
df = df[df.delivery_status.eq('delivered')]

#Cleaning outlier (Paket donasi sembako dan grosir)
## need futher data based technique
donasi = ['donasi', 'garda', 'sembako', 'psbb ']
df = df[~df['name'].str.contains('|'.join(testing))]
df = df[~(df['subtotal'] > 1500000)]
df = df[(df['harga'] > 0)]

df.info()

In [ ]:
print("Dataset dimension : " + str(df.shape))
print('Jumlah Customer : ' + str(len(pd.unique(df['telp']))))
print('Jumlah Transaksi : ' + str(len(pd.unique(df['no_order']))))

In [ ]:
df.to_csv("Hasil/Ternakmart_Transaction_CLEAN.csv")

In [ ]:
#parse datetime format
df_clean = df.copy()
df_clean['delivery_date'] = df_clean['delivery_date'].apply(lambda x:dt.datetime.strptime(x,'%d %b %Y'))
df_clean.to_csv("Hasil/Ternakmart_Transaction_CLEAN_V1.csv")
df_clean.info()

# Generate LRFM

In [ ]:
#group by invoice 
df_group = df_clean.groupby(['no_order', 'telp', 'delivery_date'], as_index = False).agg({'subtotal':'sum'})
df_group

In [ ]:
#Group By Telp
#Perform mapping to LRFM
snapshot_date = df_clean['delivery_date'].max() + timedelta(days=1)
lrfm = df_group.groupby(['telp']).agg(Length=('delivery_date', lambda x: (snapshot_date - x.min()).days),
                                   Recency=('delivery_date', lambda x: (snapshot_date - x.max()).days),
                                   Frequency=('no_order', 'count'),
                                   Monetary=('subtotal', 'sum'))
lrfm.to_csv("Hasil/Ternakmart_Transaction_LRFM.csv")
display(lrfm)
lrfm.describe()

In [ ]:
#Check Outlier on Frequency
sns.boxplot(lrfm['Frequency'])

In [ ]:
lrfm = lrfm[~(lrfm['Frequency'] > 80)]
sns.boxplot(x=lrfm['Frequency'])

In [ ]:
# Plot all 4 graphs to check skewness
plt.figure(figsize=(9, 9))

plt.subplot(4, 1, 1)
check_skew(lrfm,'Length')

plt.subplot(4, 1, 2)
check_skew(lrfm,'Recency')

plt.subplot(4, 1, 3)
check_skew(lrfm,'Frequency')

plt.subplot(4, 1, 4)
check_skew(lrfm,'Monetary')

plt.tight_layout()
plt.savefig('Grafik/before_transform.png', format='png', dpi=1000)

In [ ]:
#Frequency dan monetary skewnya ndak masuk akal

## Transformation & Normalization

In [ ]:
lrfm_trans = lrfm.copy()
#log10 Transformation
# lrfm_trans['Frequency'] = lrfm_trans['Frequency'].apply(lambda x: 1/x)
# lrfm_trans['Monetary'] = lrfm_trans['Monetary'].apply(lambda x: 1/x)

lrfm_trans['Recency'] = np.sqrt(lrfm_trans['Recency'])
lrfm_trans['Frequency'] = np.log10(lrfm_trans['Frequency']+1)
lrfm_trans['Monetary'] = np.log10(lrfm_trans['Monetary']+1)

lrfm_trans['Frequency'] = np.sqrt(lrfm_trans['Frequency'])

lrfm_trans['Frequency'] = np.sqrt(lrfm_trans['Frequency'])

plt.figure(figsize=(10, 10))
plt.subplot(4, 1, 1)
check_skew(lrfm_trans,'Length')
plt.subplot(4, 1, 2)
check_skew(lrfm_trans,'Recency')
plt.subplot(4, 1, 3)
check_skew(lrfm_trans,'Frequency')
plt.subplot(4, 1, 4)
check_skew(lrfm_trans,'Monetary')

plt.tight_layout()
plt.savefig('Grafik/after_transform.png', format='png', dpi=1000)

In [ ]:
#Min-Max Normalization
norm_lrfm = norm_minmax(lrfm_trans)
norm_lrfm.describe()

# K-Means Clustering

## Elbow Method

In [ ]:
#Elbow Method
distortions = [] 
inertias = [] 
mapping1 = {} 
mapping2 = {} 
K = range(2,10) 
  
for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(n_clusters=k).fit(norm_lrfm) 
    kmeanModel.fit(norm_lrfm)     
      
    distortions.append(sum(np.min(cdist(lrfm, kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / norm_lrfm.shape[0]) 
    inertias.append(kmeanModel.inertia_) 
  
    mapping1[k] = sum(np.min(cdist(lrfm, kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / lrfm.shape[0] 
    mapping2[k] = kmeanModel.inertia_ 

In [ ]:
kn = KneeLocator(K, distortions, curve='convex', direction='decreasing')
print("Elbow at K =", kn.knee)

plt.plot(K, distortions, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Distortion') 
plt.title('The Elbow Method using Distortion') 
plt.vlines(kn.knee, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
plt.savefig('Grafik/Elbow Method using Distortion.png', format='png', dpi=1000)
plt.show()

In [ ]:
kn = KneeLocator(K, inertias, curve='convex', direction='decreasing')
print("Elbow at K =", kn.knee)

plt.plot(K, inertias, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Inertia') 
plt.title('The Elbow Method using Inertia/SSE') 
plt.vlines(kn.knee, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
plt.savefig('Grafik/Elbow Method using Inertia.png', format='png', dpi=1000)
plt.show() 

## Silhouette Method

In [ ]:
# Silhouette Method
euclidean = []
cosine = []
mapping_euclidean = {} 
K = range(1,10)

# Prepare models
for k in K: 
    kmeans = KMeans(n_clusters=6).fit(norm_lrfm)
    normalized_vectors = preprocessing.normalize(norm_lrfm)
    normalized_kmeans = KMeans(n_clusters=4).fit(normalized_vectors)
    min_samples = norm_lrfm.shape[1]+1

    euclidean.append(silhouette_score(norm_lrfm, kmeans.labels_, metric='euclidean'))
    cosine.append(silhouette_score(normalized_vectors, normalized_kmeans.labels_, metric='cosine'))

In [ ]:
ymax = max(euclidean)
xpos = euclidean.index(ymax)
xmax = K[xpos]

print("Highest Euclidean Value = %s at K=%s" % (ymax, xmax,))
plt.plot(K, euclidean, 'bo-') 
plt.xlabel('Values of K') 
plt.ylabel('Euclidean') 
plt.title('The Silhouette Method using Euclidean Distance') 
plt.vlines(xmax, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
plt.savefig('Grafik/Silhouette Method using Euclidean Distance.png', format='png', dpi=1000)
plt.show() 

In [ ]:
ymax = max(cosine)
xpos = cosine.index(ymax)
xmax = K[xpos]

print("Highest Cosine Value : %s at K=%s" % (ymax, xmax,))
plt.plot(K, cosine, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Cosine') 
plt.title('The Silhouette Method using Cosine') 
plt.vlines(xmax, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
plt.savefig('Grafik/Silhouette Method using Cosine.png', format='png', dpi=1000)
plt.show() 

## Cluster!!

In [ ]:
def kmeans(normalised_df_lrfm, clusters_number, original_df_lrfm):
    
    kmeans = KMeans(n_clusters = clusters_number, random_state = 1)
    kmeans.fit(normalised_df_lrfm)

    # Extract cluster labels
    cluster_labels = kmeans.labels_
        
    # Create a cluster label column in original dataset
    df_new = original_df_lrfm.assign(Cluster = cluster_labels)
    
    # Initialise TSNE
    model = TSNE(random_state=1)
    transformed = model.fit_transform(df_new)
    
    # Plot t-SNE
    plt.title('Flattened Graph of {} Clusters'.format(clusters_number))
    sns.scatterplot(x=transformed[:,0], y=transformed[:,1], hue=cluster_labels, style=cluster_labels, palette="Set1")
    
    return df_new


In [ ]:
#Scatter Plot
plt.figure(figsize=(12, 15))

plt.subplot(4, 1, 1)
df_lrfm_k4 = kmeans(norm_lrfm, 4, lrfm)

plt.subplot(4, 1, 2)
df_lrfm_k5 = kmeans(norm_lrfm, 5, lrfm)

plt.subplot(4, 1, 3)
df_lrfm_k6 = kmeans(norm_lrfm, 6, lrfm)

plt.subplot(4, 1, 4)
df_lrfm_k7 = kmeans(norm_lrfm, 7, lrfm)

plt.tight_layout()
plt.savefig('Grafik/Cluster-flattened.png', format='png', dpi=300)

In [ ]:
#Snake Plot
plt.figure(figsize=(12, 15))

plt.subplot(4, 1, 1)
plt.title('Snake Plot of K-Means = 4')
snake_plot(norm_lrfm, df_lrfm_k4, lrfm)

plt.subplot(4, 1, 2)
plt.title('Snake Plot of K-Means = 5')
snake_plot(norm_lrfm, df_lrfm_k5, lrfm)

plt.subplot(4, 1, 3)
plt.title('Snake Plot of K-Means = 6')
snake_plot(norm_lrfm, df_lrfm_k6, lrfm)

plt.subplot(4, 1, 4)
plt.title('Snake Plot of K-Means = 7')
snake_plot(norm_lrfm, df_lrfm_k7, lrfm)

plt.savefig('Grafik/Cluster-snakeplot.png', format='png', dpi=300)
plt.tight_layout()

In [ ]:
#3D Plot
fig = plt.figure(figsize=(15, 20))

ax = fig.add_subplot(2, 2, 1, projection='3d')
plt.title('3D Plot of K-Means = 4')
threeD_plot(norm_lrfm, df_lrfm_k4)

ax = fig.add_subplot(2, 2, 2, projection='3d')
plt.title('3D Plot of K-Means = 5')
threeD_plot(norm_lrfm, df_lrfm_k5)

ax = fig.add_subplot(2, 2, 3, projection='3d')
plt.title('3D Plot of K-Means = 6')
threeD_plot(norm_lrfm, df_lrfm_k6)

ax = fig.add_subplot(2, 2, 4, projection='3d')
plt.title('3D Plot of K-Means = 7')
threeD_plot(norm_lrfm, df_lrfm_k7)

plt.savefig('Grafik/Cluster-3DPlot.png', format='png', dpi=300)
plt.tight_layout()

In [ ]:
#Pilih dataframe dengan jumlah cluster yang fix dipakai
df_cluster_fix = df_lrfm_k4.copy()

# Cluster Analysis

In [ ]:
#Mengubah nilai Recency menggunakan 1-R karena merupakan kebalikan dari variabel lain
#R asli jika semakin kecil akan semakin bagus
df_lrfm_all = pd.merge(df_cluster_fix, norm_lrfm, on='telp', suffixes=('_real', '_norm'))
df_lrfm_all['Recency_norm'] = 1-df_lrfm_all['Recency_norm']

df_lrfm_all.to_csv("Hasil/Ternakmart_Transaction_Clustered_LRFM.csv")
df_lrfm_all

In [ ]:
def lrfm_values(df):
    lrfm_cluster = df.groupby(['Cluster']).agg(['mean', 'min', 'max']).round(3)
    return lrfm_cluster

In [ ]:
#mean at every cluster
lrfm_values(df_lrfm_all)

In [ ]:
#mean from all dataframe
df_lrfm_all.mean(axis=0).round(3)

## Menghitung CLV

In [ ]:
#Nilai bobot didapatkan dari file excel
AHP = {
    'Length' : 0.52,
    'Recency' : 0.095,
    'Frequency' : 0.36,
    'Monetary' : 0.494}

In [ ]:
df_lrfm_all['CLV'] = (df_lrfm_all['Length_norm']*AHP['Length'] + df_lrfm_all['Recency_norm']*AHP['Recency'] + df_lrfm_all['Frequency_norm']*AHP['Frequency'] + df_lrfm_all['Monetary_norm']*AHP['Monetary'])
display(df_lrfm_all['CLV'].describe())

#Agar angka lebih cantik dan mudah dibaca, kita kalikan 1000
df_lrfm_all['CLV'] = df_lrfm_all['CLV']*100
plt.title('Distribution of CLV')
sns.distplot(df_lrfm_all['CLV'], kde=False)
plt.show()

## Rank CLV

In [ ]:
def clv_values(df):
    clv_cluster = df.groupby(['Cluster']).agg({
        'Length_real': ['min', 'max', 'mean'],
        'Recency_real': ['min', 'max', 'mean'],
        'Frequency_real': ['min', 'max', 'mean'],
        'Monetary_real': ['min', 'max', 'mean'],
        'CLV' : 'mean'
    }).round(0)
    
    return clv_cluster

In [ ]:
clv_values(df_lrfm_all).sort_values(by=[('CLV','mean')], ascending=False)

In [ ]:
#slice data berdasarkan cluster
df_lrfm_c0 = df_lrfm_all[lambda x: x['Cluster'] == 0]
df_lrfm_c1 = df_lrfm_all[lambda x: x['Cluster'] == 1]
df_lrfm_c2 = df_lrfm_all[lambda x: x['Cluster'] == 2]
df_lrfm_c3 = df_lrfm_all[lambda x: x['Cluster'] == 3]

In [ ]:
sns.boxplot(df_lrfm_c0['Length_real'])

# Market Basket Analysis

## Join Dataframe

In [ ]:
df_basket_all = pd.merge(df_clean, df_cluster_fix, on='telp', how='left')
df_basket_all = df_basket_all[['delivery_date', 'no_order', 'telp', 'prod_id', 'produk', 'qty', 'Cluster']].copy()
df_basket_all['prod_id'] = df_basket_all['prod_id'].astype(int)

#drop NA from frequency outlier
df_basket_all.dropna(subset=['Cluster'], inplace=True)

print("Dataset dimension : " + str(df_basket_all.shape))
print('Jumlah Customer : ' + str(len(pd.unique(df_basket_all['telp']))))
print('Jumlah Transaksi : ' + str(len(pd.unique(df_basket_all['no_order']))))

## Testing all Cluster

In [ ]:
#clean the product names
df_basket_all['produk'] = df_basket_all['produk'].str.strip()
df_basket_all['no_order'] = df_basket_all['no_order'].astype('str')

In [ ]:
#Split/subsetting dataframe
df_basket_cluster0 = df_basket_all[lambda x: x['Cluster'] == 0]
df_basket_cluster1 = df_basket_all[lambda x: x['Cluster'] == 1]
df_basket_cluster2 = df_basket_all[lambda x: x['Cluster'] == 2]
df_basket_cluster3 = df_basket_all[lambda x: x['Cluster'] == 3]

### Create basket datafarme from transactions data with each row representing one basket

In [ ]:
#one hot encode the basket
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

#create MBA for every cluster
def createMBA(basket_data) :
    totalTransactions = len(basket_data.index)
    totalCustomers = len(pd.unique(basket_data['telp']))
    minTransaction = totalTransactions*0.01
    min_support_calc = minTransaction/totalTransactions

    print('number of customer in cluster is', totalCustomers)
    print('number of baskets for analysis is', totalTransactions)
    print('minimum support value is ', round(min_support_calc*100, 4), '%')

    basket = basket_data.groupby(['no_order', 'produk'])['qty'].sum().unstack().reset_index().fillna(0).set_index('no_order')
    basket_sets = basket.applymap(encode_units)
    basket_sets.dropna(inplace=True)
    basket_sets = basket_sets.astype(int)
    # display(basket_sets.head(5))

    #create frequent items sets with clculated minimum support
    frequent_itemsets = fpgrowth(basket_sets, min_support=min_support_calc, use_colnames=True)
    # display(frequent_itemsets.describe())

    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0)
    rules.sort_values('support', ascending = False, inplace = True)

    createMBA.rules = rules

    return rules

In [ ]:
for i in range(4) :
    #slice data
    print("Market Basket Analysis for Cluster", i)
    basket_data = df_basket_all[lambda x: x['Cluster'] == i]

    createMBA(basket_data)

    display(createMBA.rules.head(10))
    createMBA.rules.to_csv("Hasil/Ternakmart_Results_FPGrowth_Cluster_%s.csv" % (i,), 'a')
    # rules[(rules['lift'] >= 0.2) & (rules['confidence'] >= 0.1)].sort_values(by=['confidence', 'lift'], ascending=False).to_csv("Data/Ternakmart_Results_FPGrowth_Cluster_%s.xlsx" % (i,), 'a', newline='')
    print("\n \n")